Load dependencies

In [1]:
import cobra
#Avoid the warnings about the sbml formating
import logging
logging.getLogger("cobra").setLevel(logging.ERROR)

import os
import sys
from pathlib import Path
path = Path.cwd()
sys.path.append(path)

import numpy as np

In [2]:
from reaction_class import Reaction
import gapfill_function

### Cobrapy crash tutorial for browsing GSMMs

I find cobrapy one the best ways to interact with GSMMs in a programatic way. It's quite well documented (https://cobrapy.readthedocs.io/en/latest/)

I am only going to illustrate some basic functionalities:

1) load a model from an XML file

In [3]:
model_location = os.path.join(path.parent, 'files',  'models', 'all_bins-draft.xml')
gpseq_model = cobra.io.read_sbml_model(model_location)

Academic license - for non-commercial use only - expires 2021-09-09
Using license file C:\Users\u0139894\gurobi.lic


As expected from GSMMs, the model obj has genes, reactions, metabolites, medium, demand reactions, and compartments.

They are all iterable python objects that have additional information as id, name, charge, formula, etc

In [4]:

print('number of reactions : ', len(gpseq_model.reactions))


print('number of metabolites : ', len(gpseq_model.metabolites))


print('number of genes : ', len(gpseq_model.genes))


print('Compartments : ', gpseq_model.compartments)

#1 reaction info
print('id : ', gpseq_model.reactions.rxn00004_c0.id)
print('name : ', gpseq_model.reactions.rxn00004_c0.name)

print(gpseq_model.reactions.rxn00004_c0.reaction)
print(gpseq_model.reactions.rxn00004_c0.build_reaction_string(use_metabolite_names=1))

print('\n\n', 'reaction genes : ',  gpseq_model.reactions.rxn00004_c0.genes , '\n\n')
print('\n\n', 'reaction gene rules : ', gpseq_model.reactions.rxn00004_c0.gene_reaction_rule, '\n\n',)


#1 metabolite info
print('id : ', gpseq_model.metabolites.cpd00020_c0.id)
print('name : ', gpseq_model.metabolites.cpd00020_c0.name)
print('formula : ', gpseq_model.metabolites.cpd00020_c0.formula)
print('charge : ', gpseq_model.metabolites.cpd00020_c0.charge)
print('elements : ', gpseq_model.metabolites.cpd00020_c0.elements)
print('weight : ', gpseq_model.metabolites.cpd00020_c0.formula_weight)
print('reactions that use this metabolite : ', [i.id for i in gpseq_model.metabolites.cpd00020_c0.reactions])


number of reactions :  3846
number of metabolites :  3113
number of genes :  17464
Compartments :  {'c0': '', 'e0': '', 'p0': ''}
id :  rxn00004_c0
name :  4-hydroxy-4-methyl-2-oxoglutarate pyruvate-lyase (pyruvate-forming)
cpd02570_c0 <=> 2.0 cpd00020_c0
Parapyruvate-c0 <=> 2.0 Pyruvate-c0


 reaction genes :  frozenset({<Gene gp_NODE___EIGHT____ONE____NINE____THREE___length___FOUR____ZERO____SIX____FIVE___cov___ONE____SIX____DOT____ONE____TWO____ONE____SIX____NINE____SIX_____THREE____SEVEN____EIGHT____FIVE_____THREE____THREE____ZERO____THREE__ at 0x26c79924220>, <Gene gp_NODE___ONE____THREE____SIX___length___ONE____ZERO____FOUR____SEVEN____EIGHT____FOUR___cov___THREE____ONE____DOT____SEVEN____TWO____EIGHT____SIX____EIGHT____ONE_____ONE____EIGHT____EIGHT____THREE____FOUR_____ONE____EIGHT____THREE____SEVEN____THREE__ at 0x26c79924e50>, <Gene gp_NODE___ONE____NINE____SEVEN___length___EIGHT____ONE____SIX____ZERO____FIVE___cov___EIGHT____NINE____DOT____EIGHT____NINE____ONE____THREE____SIX

All these fields are editable. 

Additionally, there is a suite of functions (check the docs).

To run FBA (the draft model is not functional)

In [5]:
solution = gpseq_model.optimize()
print(solution.objective_value)

print(solution.fluxes)

0.0
rxn00001_c0       0.0
rxn00002_c0       0.0
rxn00003_c0       0.0
rxn00004_c0       0.0
rxn00006_c0       0.0
                 ... 
EX_cpd03662_e0    0.0
EX_cpd00178_e0    0.0
EX_cpd01618_e0    0.0
bio1              0.0
DM_cpd01042_c0    0.0
Name: fluxes, Length: 3846, dtype: float64


As a quick illustration. I am going to create a reaction for ATP hydrolysis, add it to the model and use this as an objective instead of the biomass.

In [6]:
reaction = cobra.Reaction('ATHyd')
reaction.name = 'ATP hydrolysis'
atp = gpseq_model.metabolites.cpd00002_c0
water = gpseq_model.metabolites.cpd00001_c0
adp = gpseq_model.metabolites.cpd00008_c0
pi = gpseq_model.metabolites.cpd00009_c0
proton = gpseq_model.metabolites.cpd00067_c0

reaction.add_metabolites({atp:-1, water:-1, adp:1, pi:1, proton:1})

reaction.lower_bound=0
reaction.upper_bound=1000

gpseq_model.add_reactions([reaction])


gpseq_model.reactions.ATHyd.objective_coefficient=1
gpseq_model.reactions.bio1.objective_coefficient=0

In [7]:
solution = gpseq_model.optimize()
print(solution.objective_value)

print(solution.fluxes)

0.0
rxn00001_c0       0.0
rxn00002_c0       0.0
rxn00003_c0       0.0
rxn00004_c0       0.0
rxn00006_c0       0.0
                 ... 
EX_cpd00178_e0    0.0
EX_cpd01618_e0    0.0
bio1              0.0
DM_cpd01042_c0    0.0
ATHyd             0.0
Name: fluxes, Length: 3847, dtype: float64


Finally, to save the modified model as an XML file

In [8]:
new_model_location = os.path.join(path.parent, 'files',  'models', 'all_bins-draft_ATP_Obj.xml')
cobra.io.write_sbml_model(gpseq_model, new_model_location)

### Gapfilling a model on complete media with equally penalized database reactions

Use the function gapfill_function.gapfill

as inputs use:

1) The reaction ids of your draft model as a set that we call 'draft_reaction_ids'



In [11]:
model_location = os.path.join(path.parent, 'files',  'models', 'all_bins-draft_ATP_Obj.xml')
draft_model = Reaction(model=model_location)
draft_reaction_ids = set(draft_model.reactions) 

2) A Reaction class object that contains all the reactions of the ModelSeed database, called 'all_reactions' 

In [12]:
# Load the database reactions.tsv file
biochem = os.path.join(path.parent, 'files',  'biochemistry', 'reactions.tsv')
db_reactions = Reaction(biochem_input=biochem)

#Combine all reactions into one object

all_reactions = Reaction()
all_reactions.reactions = all_reactions.add_dict(draft_model.reactions, db_reactions.reactions) 

3) the id of an objective function, in our case 'bio1' or 'ATHyd'

4) A method to select the best gapfilled sets. In this case we use the minimum number of added reactions 'min_reactions'

In [13]:
model, obj, new_reacs = gapfill_function.gapfill(all_reactions, draft_reaction_ids, {}, 'bio1', result_selection = 'min_reactions')


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Delta is 70671.00
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Delta is 35335.00
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Delta is 17667.00
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Delta is 8833.00
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Delta is 4416.00
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Delta is 2208.00
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Delta is 1104.00
Warning for adding constraints: zero or small (< 1e-13) coefficients

The added reactions as listed in the 'new_reacs' list and a cobrapy model object is generated called 'model'

In [12]:
print('number of added reactions : ', len(new_reacs))
print('number of reactions in model : ', len(model.reactions))
print('number of metabolites in model : ', len(model.metabolites))

#run FBA
solution = model.optimize()
print('objective value :' , solution.objective_value)

number of added reactions :  78
number of reactions in model :  3741
number of metabolites in model :  3132
objective value : 1.0


This is still a work in progress implementation and you will notice that several features need to be added to this model to make it equal to the standard GSMMs:

- the constraints are in the range -1 and 1, while they should be in the range -1000 and 1000

In [13]:
for reaction in model.reactions:
    reaction.lower_bound*=1000
    reaction.upper_bound*=1000

solution = model.optimize()
print(solution.objective_value)

1000.0


- We are missing database information for the reactions and metabolites (name, charge, formula, pathway, genes, etc.). I will later write a function to retrieve these. 

### Gapfilling a model on a defined media

We need the same inputs as above, but now we also need a media



In [14]:
media_file = os.path.join(path.parent, 'files', 'biochemistry', 'Nitrogen-Nitrite_media.tsv')

Nit_media = {}

with open(media_file) as f:
    f.readline()
    for line in f:
        a = line.strip().split('\t')
        Nit_media['EX_' + a[0] + '_e0'] = {'lower_bound':-1, 'upper_bound':1, 'metabolites':{a[0]+'_e0':-1.0}}

In [15]:
model2, obj2, new_reacs2 = gapfill_function.gapfill(all_reactions, draft_reaction_ids, {}, 'ATHyd', result_selection = 'min_reactions', medium=Nit_media)


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Delta is 65919.00
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Delta is 32959.00
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Delta is 16479.00
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Delta is 8239.00
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Delta is 4119.00
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Delta is 2059.00
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Delta is 1029.00
Warning for adding constraints: zero or small (< 1e-13) coefficients

In [16]:
for reaction in model2.reactions:
    reaction.lower_bound*=1000
    reaction.upper_bound*=1000

print('number of added reactions : ', len(new_reacs2))
solution = model2.optimize()
print('objective value : ', solution.objective_value)

number of added reactions :  5
objective value :  1000.0


### Gapfilig a model with different costs for reactions

Now we need to provide a python dictionary with reaction costs. The function will automatically give a cost of zero to reactions that are already in the draft model and also to the exchange reactions of the defined media. It will give a default cost (set to 1) to reactions that are not in the draft model nor in the dictionary with costs.

In [17]:
#select random reaction and add random costs between 0 and 1
candidate_reactions = {}
for i in  all_reactions.reactions:
    if np.random.uniform()<0.02:
        candidate_reactions[i] = np.random.uniform()

We will also use a different criteria to choose the gapfilled reaction. Instead of 'min_reactions' we use 'min_cost', selecting the set with a minimum sum of costs.

In [ ]:
model3, obj3, new_reacs3 = gapfill_function.gapfill(all_reactions, draft_reaction_ids, candidate_reactions, 'bio1', result_selection = 'min_costs', medium=Nit_media)


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Delta is 65919.00
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Delta is 32959.00
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Delta is 16479.00
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Delta is 8239.00
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Delta is 4119.00
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Delta is 2059.00


In [26]:
for reaction in model3.reactions:
    reaction.lower_bound*=1000
    reaction.upper_bound*=1000

print('number of added reactions : ', len(new_reacs3))
solution = model3.optimize()
print(solution.objective_value)

number of added reactions :  5
1000.0


### Quick idea of what to do when no flux is observed on a particular media


1) Run a gapfill on the with complete media option (as in [5])

2) Run flux variability analysis (for big models this takes a long time to run):

In [31]:
model_exchanges = [i for i in model.reactions if ('EX_' in i.id) and ('_e0' in i.id)]

In [ ]:
fva = cobra.flux_analysis.flux_variability_analysis(model, model_exchanges)
print(fva)

The exchange reaction where the minimum and maximum are negative are likely required in the medium. They are essential for the objective and likely the database reactions have no way to make them, so gapdfilling does not wotrk.